In [ ]:
def generate_word_cloud(all_words,file_name='',bigram=True,re_color=False):
    mask = np.array(Image.open("data/hkmap.png"))
    
    font_path = os.path.join('word-cloud-font-library/','RuiZiYun.TTF')
#     'HuaWenXinWei.ttf' 
#     'chinese.simfang.ttf'

    wordcloud = WordCloud(collocations=bigram, font_path=font_path,width=2000,height=1000,
                          background_color='white',mask=mask,max_words=1000)
    wordcloud.generate(all_words)
    
    if re_color:
        wordcloud.recolor(color_func=grouped_color_func)


    figure = plt.figure(figsize=(10,8))
    
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    if file_name != '':
        save_path = os.path.join('word-cloud-image/',file_name)
        plt.savefig(save_path,dpi=500)
    plt.show()

# credit: https://amueller.github.io/word_cloud/auto_examples/colored_by_group.html
from wordcloud import (WordCloud, get_single_color_func)
class SimpleGroupedColorFunc(object):
    """Create a color function object which assigns EXACT colors
       to certain words based on the color to words mapping

       Parameters
       ----------
       color_to_words : dict(str -> list(str))
         A dictionary that maps a color to the list of words.

       default_color : str
         Color that will be assigned to a word that's not a member
         of any value from color_to_words.
    """

    def init(self, color_to_words, default_color):
        self.word_to_color = {word: color
                              for (color, words) in color_to_words.items()
                              for word in words}

        self.default_color = default_color

    def call(self, word, **kwargs):
        return self.word_to_color.get(word, self.default_color)

class GroupedColorFunc(object):
    """Create a color function object which assigns DIFFERENT SHADES of
       specified colors to certain words based on the color to words mapping.

       Uses wordcloud.get_single_color_func

       Parameters
       ----------
       color_to_words : dict(str -> list(str))
         A dictionary that maps a color to the list of words.

       default_color : str
         Color that will be assigned to a word that's not a member
         of any value from color_to_words.
    """

    def init(self, color_to_words, default_color):
        self.color_func_to_words = [
            (get_single_color_func(color), set(words))
            for (color, words) in color_to_words.items()]

        self.default_color_func = get_single_color_func(default_color)

    def get_color_func(self, word):
        """Returns a single_color_func associated with the word"""
        try:
            color_func = next(
                color_func for (color_func, words) in self.color_func_to_words
                if word in words)
        except StopIteration:
            color_func = self.default_color_func

        return color_func

    def call(self, word, **kwargs):
        return self.get_color_func(word)(word, **kwargs)

color_to_words = {
    # words below will be colored with a green single color function
    '#00ff00': topic_sentiment_result_proportion_df[topic_sentiment_result_proportion_df.dominant=='pos'].index,
    # will be colored with a red single color function
    'red': topic_sentiment_result_proportion_df[topic_sentiment_result_proportion_df.dominant=='neg'].index
}
default_color = 'grey'

grouped_color_func = GroupedColorFunc(color_to_words, default_color)